In [50]:
import numpy as np
import pandas as pd
import json
from epilepsypcm.utils.outcome_params import node_location, early_propogation, irritative_zone

from epilepsypcm.utils.make_df import make_df, get_df_list, concat_dfs
from epilepsypcm.models.base_models import *
from sklearn.metrics import auc


all_positive_patients = pd.read_csv("df_final.csv", sep = ',').reset_index(drop = True)
print(all_positive_patients["outcome"].value_counts())


0.0    259
1.0     56
Name: outcome, dtype: int64


In [2]:
all_positive_patients.head()

,Unnamed: 0,Channels,outcome,IZ,EP,SigResp,SigStim,N1RespAvg,N1RespSDV,N2RespAvg,...,N1StimSDV,N2StimAvg,N2StimSDV,P2StimAvg,P2StimSDV,patient,InDegree,OutDegree,EV,Closeness
0,0,LA1_LA2,1.0,0.0,0.0,0.000000,0.064516,0.000000,0.000000,0.000000,...,2.011706,3.724259,3.460362,2.796966,3.000218,PY21N008,0.000000,0.071429,4.993936e-14,0.000000
1,1,LA9_LA10,0.0,1.0,0.0,0.000000,0.193548,1.808045,1.676081,2.493333,...,9.957544,5.060794,6.246739,4.478926,6.889325,PY21N008,0.000000,0.214286,4.993936e-14,0.000000
2,2,LAH1_LAH2,1.0,1.0,0.0,0.064516,0.193548,7.116911,19.154270,4.252294,...,14.745868,3.809967,3.367630,1.821021,1.867067,PY21N008,0.071429,0.214286,7.960633e-04,0.174603
3,3,LAH8_LAH9,0.0,0.0,0.0,0.290323,0.096774,4.306443,3.856300,4.059026,...,2.173960,1.695722,1.598789,1.574253,2.044144,PY21N008,0.321429,0.107143,1.590588e-01,0.480159
4,4,LPH1_LPH2,0.0,1.0,0.0,0.096774,0.258065,5.792531,15.863221,3.143365,...,7.764073,3.585250,3.693649,3.242192,3.639533,PY21N008,0.107143,0.285714,4.743272e-03,0.218807


In [3]:
all_positive_patients["patient"].value_counts()

PY21N004    46
PY21N002    39
PY21N008    31
PY18N015    24
PY19N026    23
PY18N013    20
PY21N006    20
PY18N016    19
PY20N001    17
PY19N012    17
PY18N002    16
PY19N023    14
PY16N008     9
PY19N009     8
PY17N005     5
PY17N008     4
PY16N013     3
Name: patient, dtype: int64

In [13]:
#scaling original data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

all_positive_patients_norm = all_positive_patients

patient_list = all_positive_patients["patient"]
all_positive_patients_norm = all_positive_patients_norm.drop(['Channels', "patient"], axis = 1)
all_positive_patients_norm =  pd.DataFrame(scaler.fit_transform(all_positive_patients_norm.to_numpy()))
all_positive_patients_norm["patient"] = patient_list

In [44]:
patient_df = all_positive_patients_norm.groupby(by = ["patient"]).mean()
patient_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
patient,,,,,,,,,,,,,,,,,,,,,
PY16N008,0.987261,0.333333,0.333333,0.000000,0.195602,0.200617,0.164193,0.104672,0.109099,0.080189,...,0.167691,0.101105,0.115796,0.061533,0.168954,0.072589,0.240741,0.240741,0.405874,0.322986
PY16N013,0.770701,0.000000,0.333333,0.000000,0.270833,0.277778,0.360883,0.189456,0.266252,0.249138,...,0.368573,0.195130,0.340871,0.235600,0.059010,0.018300,0.666667,0.666667,0.666667,0.666667
PY17N005,0.487261,0.200000,0.000000,0.000000,0.487500,0.500000,0.277625,0.161090,0.219309,0.208649,...,0.283540,0.152627,0.273639,0.141840,0.292992,0.099758,0.500000,0.500000,0.601284,0.564286
PY17N008,0.893312,0.000000,0.750000,0.750000,0.130580,0.117188,0.046634,0.020204,0.051603,0.048049,...,0.045184,0.018858,0.029611,0.032682,0.045048,0.013629,0.500000,0.500000,0.500000,0.500000
PY18N002,0.800955,0.312500,0.187500,0.250000,0.069643,0.087484,0.081150,0.086667,0.084885,0.095804,...,0.084327,0.083009,0.085656,0.086904,0.052315,0.033880,0.104167,0.104167,0.168034,0.121466
PY18N013,0.447452,0.000000,0.000000,0.000000,0.246652,0.246875,0.152019,0.098721,0.210023,0.233996,...,0.154037,0.097446,0.258714,0.232430,0.183782,0.084382,0.307692,0.307692,0.240715,0.389389
PY18N015,0.377389,0.208333,0.000000,0.416667,0.170625,0.170356,0.152704,0.145168,0.167845,0.241276,...,0.157576,0.142686,0.205918,0.207137,0.168822,0.114612,0.150362,0.150362,0.228299,0.296006
PY18N016,0.856688,0.210526,0.421053,0.210526,0.320724,0.328947,0.203026,0.106426,0.245223,0.207945,...,0.207352,0.109398,0.310754,0.189003,0.170511,0.071483,0.277778,0.277778,0.251017,0.509004
PY19N009,0.227707,0.250000,0.500000,0.250000,0.323730,0.332031,0.174030,0.096421,0.126919,0.115013,...,0.177738,0.087405,0.141318,0.093857,0.216665,0.098211,0.303571,0.303571,0.481218,0.334821


In [45]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components = 3)
components = pca.fit_transform(patient_df)
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

patient_df = patient_df.reset_index()

In [46]:
pca_df = pd.DataFrame(components)
pca_df.columns = ["PC1", "PC2", "PC3"]

In [49]:
fig = px.scatter_3d(
    pca_df,
    x = "PC1",
    y = "PC2",
    z = "PC3",
    #labels=labels,
    color=patient_df["patient"]
)

fig.show()